# Auto Generated Agent Chat: Society of Minds

This notebook showcases how to use conversable agents in AutoGen to realize a Society of Mind (SoM) scenario, in which an agent runs a  chat session with another agent as an inner-monologue. The idea of SoM and part of the example demonstrated in this notebook come from this PR: https://github.com/microsoft/autogen/pull/890
 

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

## Society of Minds

In [7]:
import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
llm_config = {"config_list": config_list}

financial_tasks = [
    """On which days in 2024 was Microsoft Stock higher than $370? Put results in a table and don't use ``` ``` to include table.""",
    """Investigate the possible reasons of the stock performance.""",
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

### Construct a group chat manager to be used for the inner-monologue

In [8]:
assistant = autogen.AssistantAgent(
    "Inner-assistant",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

code_interpreter = autogen.UserProxyAgent(
    "Inner-code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

groupchat = autogen.GroupChat(
    agents=[assistant, code_interpreter],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
)

### Example 1: Solve one task with SoM

In [9]:
financial_assistant_1 = autogen.AssistantAgent(
    name="Financial_assistant_1",
    llm_config={"config_list": config_list},
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

financial_assistant_1.register_reply(
    [autogen.Agent, None], autogen.nested_chat_reply(manager, takeaway_method="llm"), 3
)

user.initiate_chat(financial_assistant_1, message=financial_tasks[0])

User (to Financial_assistant_1):

On which days in 2024 was Microsoft Stock higher than $370? Put results in a table and don't use ``` ``` to include table.

--------------------------------------------------------------------------------
Financial_assistant_1 (to chat_manager):

On which days in 2024 was Microsoft Stock higher than $370? Put results in a table and don't use ``` ``` to include table.

--------------------------------------------------------------------------------
Inner-assistant (to chat_manager):

To determine on which days in 2024 Microsoft's stock (MSFT) price was higher than $370, we will need historical stock price data. This information can be obtained through various financial data APIs such as Yahoo Finance, Alpha Vantage, or others. We can write a Python script to fetch the data and filter out the dates when Microsoft's stock price was higher than $370.

I will provide you with a Python script that uses the `yfinance` library, which is a popular choice that c

/Users/qingyunwu/Documents/github/autogen/autogen/agentchat/conversable_agent.py:773: UserWarning: No takeaway_method provided or takeaway_method is not supported: 
  warnings.warn("No takeaway_method provided or takeaway_method is not supported: ")


''

### Example 2: Solve multiple tasks with SoM

In [10]:
financial_assistant_2 = autogen.AssistantAgent(
    name="Financial_assistant_2",
    llm_config={"config_list": config_list},
)
writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

financial_assistant_2.register_reply(
    [autogen.Agent, None], autogen.nested_chat_reply(manager, takeaway_method="llm"), 3
)
user.initiate_chats(
    [
        {
            "recipient": financial_assistant_1,
            "message": financial_tasks[0],
            "clear_history": True,
            "silent": False,
            "takeaway_method": "llm",
        },
        {
            "recipient": financial_assistant_2,
            "message": financial_tasks[1],
            "takeaway_method": "llm",
        },
        {
            "recipient": writer,
            "message": writing_tasks[0],
        },
    ]
)


********************************************************************************
Start work on the following task: 
On which days in 2024 was Microsoft Stock higher than $370? Put results in a table and don't use ``` ``` to include table.

With the following carryover from previous tasks: 


********************************************************************************
User (to Financial_assistant_1):

On which days in 2024 was Microsoft Stock higher than $370? Put results in a table and don't use ``` ``` to include table.

--------------------------------------------------------------------------------
Financial_assistant_1 (to chat_manager):

On which days in 2024 was Microsoft Stock higher than $370? Put results in a table and don't use ``` ``` to include table.

--------------------------------------------------------------------------------
Inner-assistant (to chat_manager):

To determine on which days in 2024 Microsoft's stock (MSFT) price was higher than $370, we will need hi

/Users/qingyunwu/Documents/github/autogen/autogen/agentchat/conversable_agent.py:773: UserWarning: No takeaway_method provided or takeaway_method is not supported: 
  warnings.warn("No takeaway_method provided or takeaway_method is not supported: ")
